# ML 7:   Unsupervised learning - K-means Clustering

Objectives: Implement K-means clustering algorithm and apply it for image compression. 

In [1]:
#Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 

### Load and plot Data
You will ﬁrst start on a 2D dataset example that will help you gain an intuition of how K-means algorithm works.

Plot the data and get a figure similar to Fig.1. 

<img src="images/f2.png" style="width:350px;height:250px;">
<caption><center> **Fig. 1** : **Training data** </center></caption>

In [ ]:
# Use loadmat to load matlab file ex7data2.mat and extact X

X = ?

#Create Fig.1.

 ?

### K-means algorithm

K-means is a method to automatically cluster similar data examples together. You have a training set ${x^{(1)} , ..., x^{(m)} }$, where $x^{(i)} ∈ R^n $, and want to group the data into a few cohesive clusters. The intuition behind K-means is an iterative procedure that starts by guessing the initial centroids, and then reﬁnes this guess by repeatedly assigning examples to their closest centroids and then re-computing the centroids based on the assignments. 

The algorithm repeatedly carries out two steps: 

(i) Assigning each training example x(i) to its closest centroid

(ii) Re-computing the mean of each centroid using the points assigned to it. 

K-means algorithm will always converge to some ﬁnal set of centroids. Note that the converged solution may not always be ideal and depends on the initial setting of the centroids. Therefore, in practice K-means usually runs a few times with different random initializations. One way to choose the best clustering solution is to choose the one with the lowest cost function value (distortion). 

### Finding closest centroids
In the K-means cluster assignment phase, the algorithm assigns every training example $x^{(i)}$ to its closest centroid, given the current positions of the centroids. Speciﬁcally, for every example $x^{(i)}$ we set

$c^{(i)} := j$ that minimizes $\|x^{(i)} - u_j\|^2$

where $c^{(i)}$ is the index of the centroid that is closest to $x^{(i)}$, and $u_j$ is the position of the j’th centroid. Note that $c^{(i)}$ corresponds to idx[i] in the code below.

Function *findClosestCentroids* takes the data matrix X and the locations of all centroids and outputs a one-dimensional array idx that holds the index of the closest centroid to every training example. The index is an integer value in {1, ..., K}, where K is the number of centroids. This is implemented using a loop over every training example. 

Complete *findClosestCentroids* and run it to see the centroid assignments for the ﬁrst 3 examples.


In [ ]:
def findClosestCentroids(X, centroids):
    """
    Returns the closest centroids in idx for a dataset X where each row is a single example.
    """
    K = ? #number of clusters
    idx = np.zeros((X.shape[0],1))
    temp = np.zeros((centroids.shape[0],1))
    
    for i in range(X.shape[0]):
        for j in range(K):
            dist = X[i,:] - centroids[j,:]
            temp[j] = np.sum(dist**2)
        idx[i] = np.argmin(temp)+1
    return idx

In [ ]:
# call function findClosestCentroids
K = 3
initial_centroids = np.array([[3,3],[6,2],[8,5]])
idx = findClosestCentroids(X, initial_centroids)
print("Closest centroids for the first 3 examples:\n",idx[0:3])  #ANSWER  [1 3 2]

### Computing centroid means

Given assignments of every point to a centroid, the second phase of the algorithm re-computes, for each centroid, the mean of the points that were assigned to it. Speciﬁcally, for every centroid k we set 

$u_k := \frac{1}{C_k} \Sigma_{i\in C_k}x^{(i)}$

where ${C_k}$ is the set of examples assigned to centroid k. Concretely, if two examples say $x^{(3)}$ and $x^{(5)}$ are assigned to centroid k=2, then you should update $u_2=0.5(x^{(3)} + x^{(5)})$. Complete function *computeCentroids* and run it to output the new positions of the centroids after the ﬁrst step of K-means.


In [ ]:
def computeCentroids(X, idx, K):
    """
    K- number of clusters; X -data matrix; 
    idx - index of the closest centroid to every training example. 
    
    Returns the new centroids by computing the means of the data points assigned to each centroid.
    """
    m = ? # Number of examples 
    n = ? # Number of features
    
    #Inicialize the centroids by 0
    centroids = ?
    count = np.zeros((K,1))
    
    for i in range(m):
        index = int((idx[i]-1)[0])
        centroids[index,:]+=X[i,:]
        count[index]+=1
    
    return centroids/count

In [ ]:
# call function computeCentroids and compute centroids
centroids = ?
print("Centroids computed after initial finding of closest centroids:\n", centroids)

### Random Initialization

In practice, a good strategy for initializing the centroids is to select random examples from the training set. 

Complete the function *kMeansInitCentroids*. 

In [ ]:
def kMeansInitCentroids(X, K):
    """
    This function initializes K centroids to be used in K-Means on dataset X
    """
    m = ? # Number of examples 
    n = ? # Number of features
    
    #Inicialize the centroids by 0
    centroids = ?
    
    for i in range(K):
        centroids[i] = X[np.random.randint(0,m+1),:]
        
    return centroids

### Run and Visualize K-means 

Now you have the two functions (*findClosestCentroids* and *computeCentroids*) completed, the next step is to run the K-means algorithm on a toy 2D dataset. The two functions are called from inside *runKmeans* function. 

In [ ]:
def plotKmeans(X, centroids, K, num_iters):
    """
    plots the data points with colors assigned to each centroid
    """
    m = ? # Number of examples 
    n = ? # Number of features
    
    
    fig, ax = plt.subplots(nrows=num_iters,ncols=1,figsize=(6,36))
    
    for i in range(num_iters):    
        
        # assign each training example to the nearest centroid
        idx = ?
                
        # Compute the centroids mean
        centroids = ?
        
        # Visualisation of data
        color = "rgb"
        for k in range(1,K+1):
            grp = (idx==k).reshape(m,1)
            ax[i].scatter(X[grp[:,0],0],X[grp[:,0],1],c=color[k-1],s=15)

        # visualize the new centroids
        ax[i].scatter(centroids[:,0],centroids[:,1],s=120,marker="x",c="black",linewidth=3)
        title = "Iteration Number " + str(i)
        ax[i].set_title(title) 

    
    plt.tight_layout()

In [ ]:
initial_centroids = kMeansInitCentroids(X, K)
plotKmeans(X, initial_centroids, K, 11)

### Image Compression with K-means

Now, you will apply K-means to image compression. In a 24-bit color representation of an image, each pixel is represented by an 8-bit unsigned integers (ranging from 0 to 255) that specify the red, green and blue intensity values (RGB encoding). The image contains thousands of colors, and now, you will reduce the number of colors to 16. By making this reduction, it is possible to represent the photo in a compressed form. Speciﬁcally, you only need to store the RGB values of the 16 selected colors, and for each pixel in the image you now need to only store the index of the color at that location (where only 4 bits are necessary to represent 16 colors). 

In this exercise, you will use K-means to select the 16 colors that will be used to represent the compressed image. Concretely, you will treat every pixel in the original image as a data example and use K-means to ﬁnd the 16 colors that best group (cluster) the pixels in the 3-dimensional RGB space. Once you have computed the cluster centroids on the image, you will then use the 16 colors to replace the pixels in the original image.

The code ﬁrst loads the image, and then reshapes it to create a 2-D matrix X2 (mx3) of the pixels, where m=16384= 128×128, the number of pixels for each RGB color, and calls the K-means function on it. After ﬁnding the top K=16 colors to represent the image, each pixel is substituted by its closest centroid. This allows to represent the original image using the centroid assignments of each pixel. Notice that you have signiﬁcantly reduced the number of bits that are required to describe the image. The original image required 24 bits (3*8) for each one of the 128×128 pixels, resulting in total size of 128 × 128 × 24 = 393216 bits. The new representation requires 16 colors (i.e. 4 bits per pixel). The ﬁnal number of bits used is therefore 128×128×4=65920 bits, which corresponds to compressing the original image by about a factor of 6.

In [ ]:
# Use loadmat to load  the image A
 
A = ?
#Check what is the shape of image A
?

#Normalize all pixels (/255)

#reshape A to one long vector of all pixels for each chanel (RGB)

X2 = ?  #X2.shape= # of pixels x # of chanels (RGB)


In [ ]:
def runKmeans(X, initial_centroids,num_iters,K):
     
    for i in range(num_iters):
                
        # assign each training example to the nearest centroid
        idx =  ?
        
        # Compute the centroids mean
        centroids =  ?
       
    return centroids, idx

In [ ]:
# Running K-means algorithm on the data
K2 = 16
num_iters = 10

initial_centroids2 =  ?
centroids2, idx2 = ?

You can now view the eﬀects of the compression by reconstructing the image based only on the centroid assignments.Fig. 1 shows the original and compressed images. Even though the resulting image retains most of the characteristics of the original, we also see compression artifacts.


<img src="images/f1.png" style="width:350px;height:200px;">
<caption><center> **Fig.1** : **Original and reconstructed image (using K-means to compress the image)** </center></caption>

In [ ]:
#X2_recovered = X2.copy()
X2_recovered = X2
for i in range(1,K2+1):
    X2_recovered[(idx2==i).ravel(),:] = centroids2[i-1]

# Display the images
import matplotlib.image as mpimg
fig, ax = plt.subplots(1,2)
ax[0].imshow(A)
ax[0].set_title("Original image") 

# Reshape the recovered image into proper dimensions
ax[1].imshow(X2_recovered.reshape(128,128,3))
ax[1].set_title("Compressed image (with 16 colors)") 

#### Use your own image (optional)

You may modify the code to run on your own image. If your image is very large, then K-means can take a long time to run. We recommend that you resize your images to manageable sizes before running the code. You can also try to vary K to see the eﬀects on the compression.
